# 🔢 Exploration du Dataset MNIST

Ce notebook explore le célèbre dataset MNIST de chiffres manuscrits.

## 1. Chargement des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Chargement du dataset depuis Hugging Face
splits = {
    'train': 'mnist/train-00000-of-00001.parquet',
    'test': 'mnist/test-00000-of-00001.parquet'
}

df_train = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["test"])

print(f"✅ Données chargées avec succès!")
print(f"📊 Taille du set d'entraînement: {len(df_train)} images")
print(f"📊 Taille du set de test: {len(df_test)} images")

## 2. Exploration des données

In [ ]:
# Structure du DataFrame
print("Colonnes du dataset:")
print(df_train.columns.tolist())
print("\nAperçu des premières lignes:")
df_train.head()

In [ ]:
# Distribution des labels
print("Distribution des chiffres dans le set d'entraînement:")
label_counts = df_train['label'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
plt.bar(label_counts.index, label_counts.values, color='steelblue', edgecolor='black')
plt.xlabel('Chiffre', fontsize=12)
plt.ylabel('Nombre d\'images', fontsize=12)
plt.title('Distribution des chiffres dans MNIST (train)', fontsize=14)
plt.xticks(range(10))
for i, v in enumerate(label_counts.values):
    plt.text(i, v + 100, str(v), ha='center', fontsize=9)
plt.tight_layout()
plt.show()

## 3. Visualisation des images

In [ ]:
def extract_image(row):
    """Extrait l'image depuis la colonne 'image' du DataFrame."""
    img_data = row['image']
    if isinstance(img_data, dict) and 'bytes' in img_data:
        # Format avec bytes
        from io import BytesIO
        img = Image.open(BytesIO(img_data['bytes']))
        return np.array(img)
    elif isinstance(img_data, Image.Image):
        return np.array(img_data)
    elif isinstance(img_data, np.ndarray):
        return img_data
    else:
        # Essayer de convertir directement
        return np.array(img_data)

# Afficher quelques exemples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Exemples d\'images MNIST', fontsize=14)

for idx, ax in enumerate(axes.flat):
    img = extract_image(df_train.iloc[idx])
    label = df_train.iloc[idx]['label']
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Label: {label}', fontsize=11)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Afficher un exemple de chaque chiffre
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Un exemple de chaque chiffre (0-9)', fontsize=14)

for digit in range(10):
    ax = axes[digit // 5, digit % 5]
    sample = df_train[df_train['label'] == digit].iloc[0]
    img = extract_image(sample)
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Chiffre: {digit}', fontsize=11)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 4. Préparation des données pour le Machine Learning

In [ ]:
# Convertir toutes les images en arrays numpy
print("Conversion des images en arrays numpy...")

X_train = np.array([extract_image(row) for _, row in df_train.iterrows()])
y_train = df_train['label'].values

X_test = np.array([extract_image(row) for _, row in df_test.iterrows()])
y_test = df_test['label'].values

print(f"\n✅ Conversion terminée!")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Normalisation des données (0-1)
X_train_norm = X_train.astype('float32') / 255.0
X_test_norm = X_test.astype('float32') / 255.0

# Aplatir les images pour les modèles classiques (28x28 -> 784)
X_train_flat = X_train_norm.reshape(X_train_norm.shape[0], -1)
X_test_flat = X_test_norm.reshape(X_test_norm.shape[0], -1)

print(f"Données normalisées et aplaties:")
print(f"X_train_flat shape: {X_train_flat.shape}")
print(f"X_test_flat shape: {X_test_flat.shape}")

## 5. Modèle simple de classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Entraînement d'un modèle de régression logistique
print("🔄 Entraînement du modèle de régression logistique...")
print("(Cela peut prendre quelques minutes)\n")

model = LogisticRegression(max_iter=100, solver='lbfgs', multi_class='multinomial', n_jobs=-1)
model.fit(X_train_flat, y_train)

print("✅ Entraînement terminé!")

In [ ]:
# Évaluation du modèle
y_pred = model.predict(X_test_flat)
accuracy = accuracy_score(y_test, y_pred)

print(f"🎯 Précision sur le set de test: {accuracy:.4f} ({accuracy*100:.2f}%)\n")
print("Rapport de classification:")
print(classification_report(y_test, y_pred))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Vraie valeur', fontsize=12)
plt.title('Matrice de confusion - MNIST', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Visualisation des prédictions

In [ ]:
# Afficher quelques prédictions
fig, axes = plt.subplots(3, 5, figsize=(14, 8))
fig.suptitle('Exemples de prédictions', fontsize=14)

indices = np.random.choice(len(X_test), 15, replace=False)

for i, (ax, idx) in enumerate(zip(axes.flat, indices)):
    ax.imshow(X_test[idx], cmap='gray')
    pred = y_pred[idx]
    true = y_test[idx]
    color = 'green' if pred == true else 'red'
    ax.set_title(f'Préd: {pred} | Vrai: {true}', color=color, fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Afficher les erreurs
errors = np.where(y_pred != y_test)[0]
print(f"Nombre d'erreurs: {len(errors)} sur {len(y_test)} ({len(errors)/len(y_test)*100:.2f}%)\n")

# Afficher quelques erreurs
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
fig.suptitle('Exemples d\'erreurs de classification', fontsize=14)

for i, ax in enumerate(axes.flat):
    if i < len(errors):
        idx = errors[i]
        ax.imshow(X_test[idx], cmap='gray')
        ax.set_title(f'Préd: {y_pred[idx]} | Vrai: {y_test[idx]}', color='red', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 7. Analyse des pixels moyens par chiffre

In [ ]:
# Calculer l'image moyenne pour chaque chiffre
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Image moyenne pour chaque chiffre', fontsize=14)

for digit in range(10):
    ax = axes[digit // 5, digit % 5]
    mask = y_train == digit
    mean_img = X_train[mask].mean(axis=0)
    ax.imshow(mean_img, cmap='hot')
    ax.set_title(f'Chiffre: {digit}', fontsize=11)
    ax.axis('off')

plt.tight_layout()
plt.show()

---
## 📝 Résumé

Dans ce notebook, nous avons:
1. Chargé le dataset MNIST depuis Hugging Face
2. Exploré la structure et la distribution des données
3. Visualisé des exemples d'images
4. Préparé les données pour le machine learning
5. Entraîné un modèle de régression logistique simple
6. Évalué les performances du modèle
7. Analysé les images moyennes par chiffre

**Prochaines étapes possibles:**
- Essayer d'autres modèles (SVM, Random Forest, KNN)
- Implémenter un réseau de neurones avec TensorFlow/PyTorch
- Appliquer des techniques d'augmentation de données
- Explorer la réduction de dimensionnalité (PCA, t-SNE)